In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
from Qommunity.searchers.hierarchical_searcher import HierarchicalSearcher

In [39]:
G = nx.powerlaw_cluster_graph(n=100, m=1, p=0.1)
num_reads = 100
version = ""
region = "na-west-1"

### Clique embedding

In [40]:
advantage = AdvantageSampler(
    G, num_reads=num_reads, version=version, region=region, use_clique_embedding=True, measure_times=True, return_sampleset_info=True
)

In [5]:
hierarchical_searcher = HierarchicalSearcher(advantage)

In [6]:
res = hierarchical_searcher.hierarchical_community_search(samples_filename="adv_clique_samples_2", division_tree=True, return_modularities=True, return_sampleset_info = True)

In [9]:
result = res

In [10]:
res, division_tree, division_modularities, sampleset_info = result

In [11]:
print(f"Modularity: {nx.community.modularity(G, res)}")

Modularity: 0.7546678910315275


In [19]:
sampleset_info[0].dwave_sampleset_info

rec.array((24068., 20., 200.1, 39832.76, 1443.24, 15764.76, 20.58, 32., 32.),
          dtype=[('qpu_sampling_time', '<f8'), ('qpu_anneal_time_per_sample', '<f8'), ('qpu_readout_time_per_sample', '<f8'), ('qpu_access_time', '<f8'), ('qpu_access_overhead_time', '<f8'), ('qpu_programming_time', '<f8'), ('qpu_delay_time_per_sample', '<f8'), ('post_processing_overhead_time', '<f8'), ('total_post_processing_time', '<f8')])

In [21]:
sampleset_info[0].dwave_sampleset_info

rec.array((24068., 20., 200.1, 39832.76, 1443.24, 15764.76, 20.58, 32., 32.),
          dtype=[('qpu_sampling_time', '<f8'), ('qpu_anneal_time_per_sample', '<f8'), ('qpu_readout_time_per_sample', '<f8'), ('qpu_access_time', '<f8'), ('qpu_access_overhead_time', '<f8'), ('qpu_programming_time', '<f8'), ('qpu_delay_time_per_sample', '<f8'), ('post_processing_overhead_time', '<f8'), ('total_post_processing_time', '<f8')])

In [25]:
sampleset_info[0].dwave_sampleset_info.qpu_access_time

array(39832.76)

In [26]:
sampleset_info[0].dwave_sampleset_info.qpu_access_time

array(39832.76)

In [7]:
from Qommunity.samplers.hierarchical.hierarchical_sampler import (
    HierarchicalSampler,
)
from Qommunity.searchers.hierarchical_searcher import (
    HierarchicalSearcher,
)
import networkx as nx
from time import time
from tqdm import tqdm
import numpy as np
import warnings


class MethodArgsWarning(Warning):
    def __init__(self, msg):
        super().__init__(msg)


# Warning format compatible with tqdm
def warn(message, category, filename, lineno, file=None, line=None):
    tqdm.write(f"Warning: {str(message)}")


warnings.showwarning = warn
warnings.simplefilter("always", MethodArgsWarning)


class IterativeHierarchicalSearcher:
    def __init__(self, sampler: HierarchicalSampler) -> None:
        self.sampler = sampler
        self.searcher = HierarchicalSearcher(self.sampler)

    def _default_saving_path(self) -> str:
        return (
            f"{self.sampler.__class__.__name__}"
            + "-network_size_"
            + f"{self.sampler.G.number_of_nodes()}"
        )

    def _verify_kwargs(self, kwargs) -> dict:
        kwargs_unhandled = ["division_tree", "return_modularities"]
        kwargs_warning = []
        for kwarg in kwargs_unhandled:
            if kwarg in kwargs:
                kwargs.pop(kwarg, None)
                kwargs_warning.append(kwarg)
        if kwargs_warning:
            msg = ", ".join(kwargs_warning)
            warnings.warn(
                f"in order to get {msg} run "
                + " IterativeSearcher.run_with_sampleset_info()"
            )

        return kwargs

    def run(
        self,
        num_runs: int,
        save_results: bool = True,
        saving_path: str | None = None,
        elapse_times: bool = True,
        iterative_verbosity: int = 0,
        return_sampleset_info: bool = False,
        **kwargs,
    ):
        kwargs = self._verify_kwargs(kwargs)

        if iterative_verbosity >= 1:
            print("Starting community detection iterations")

        if save_results and saving_path is None:
            saving_path = self._default_saving_path()

        modularities = np.zeros((num_runs))
        communities = np.empty((num_runs), dtype=object)
        times = np.zeros((num_runs))
        sampleset_infos = np.empty((num_runs), dtype=object)

        for iter in tqdm(range(num_runs)):
            elapsed = time()
            result = self.searcher.hierarchical_community_search(**kwargs)
            times[iter] = time() - elapsed

            if "return_sampleset_info" in kwargs:
                result, sampleset_info = result

            try:
                modularity_score = nx.community.modularity(
                    self.searcher.sampler.G,
                    result,
                    resolution=self.sampler.resolution,
                )
            except Exception as e:
                print(f"iteration: {iter} exception: {e}")
                modularity_score = -1

            communities[iter] = result
            modularities[iter] = modularity_score
            sampleset_infos[iter] = sampleset_info

            if save_results:
                np.save(f"{saving_path}_modularities", modularities)
                np.save(f"{saving_path}_communities", communities)
                if elapse_times:
                    np.save(f"{saving_path}_times", times)
                if return_sampleset_info:
                    np.save(
                        f"{saving_path}_sampleset_infos", sampleset_infos
                    )

            if iterative_verbosity >= 1:
                print(f"Iteration {iter} completed")

        if elapse_times and sampleset_info:
            return communities, modularities, times, sampleset_info
        if elapse_times:
            return communities, modularities, times
        if return_sampleset_info:
            return communities, modularities, sampleset_infos
        return communities, modularities

    def run_with_sampleset_info(
        self,
        num_runs: int,
        save_results: bool = True,
        saving_path: str | None = None,
        iterative_verbosity: int = 0,
        return_sampleset_info: bool = True,
        **kwargs,
    ):

        if iterative_verbosity >= 1:
            print("Starting community detection iterations")

        if save_results and saving_path is None:
            saving_path = self._default_saving_path()

        modularities = np.zeros((num_runs))
        communities = np.empty((num_runs), dtype=object)
        times = np.zeros((num_runs))
        division_modularities = np.empty((num_runs), dtype=object)
        division_trees = np.empty((num_runs), dtype=object)
        sampleset_infos = np.empty((num_runs), dtype=object)

        if return_sampleset_info:
            kwargs["return_sampleset_info"] = True

        for iter in tqdm(range(num_runs)):
            elapsed = time()
            result = self.searcher.hierarchical_community_search(
                return_modularities=True,
                division_tree=True,
                **kwargs,
            )
            if return_sampleset_info:
                (
                    communities_result,
                    div_tree,
                    div_modularities,
                    sampleset_info,
                ) = result
            else:
                (
                    communities_result,
                    div_tree,
                    div_modularities,
                ) = result
            times[iter] = time() - elapsed
            division_trees[iter] = div_tree
            division_modularities[iter] = div_modularities
            sampleset_infos[iter] = sampleset_info

            try:
                modularity_score = nx.community.modularity(
                    self.searcher.sampler.G,
                    communities_result,
                    resolution=self.sampler.resolution,
                )
            except Exception as e:
                print(f"iteration: {iter} exception: {e}")
                modularity_score = -1

            communities[iter] = communities_result
            modularities[iter] = modularity_score

            if save_results:
                np.save(f"{saving_path}_modularities", modularities)
                np.save(f"{saving_path}_communities", communities)
                np.save(f"{saving_path}_times", times)
                np.save(f"{saving_path}_division_trees", division_trees)
                np.save(
                    f"{saving_path}_division_modularities",
                    division_modularities,
                )
                np.save(
                    f"{saving_path}_sampleset_infos", sampleset_infos
                )

            if iterative_verbosity >= 1:
                print(f"Iteration {iter} completed")

        from numpy.lib import recfunctions as rfn

        dtypes = [
            ("communities", object),
            ("modularity", np.float_),
            ("time", np.float_),
            ("division_tree", object),
            ("division_modularities", object),
        ]
        sampleset_components = [
            communities,
                modularities,
                times,
                division_trees,
                division_modularities,
        ]
        
        if return_sampleset_info:
            dtypes.append(("sampleset_infos", object))
            sampleset_components.append(sampleset_infos)
        
        sampleset = np.rec.fromarrays(
            sampleset_components,
            dtype=dtypes,
        )
        
        return sampleset


In [ ]:
process_results = 

In [8]:
iterative_searcher = IterativeHierarchicalSearcher(advantage)

In [9]:
res = iterative_searcher.run_with_sampleset_info(
    num_runs=2,
    save_results=True,
    saving_path="iterative_advantage_clique_samples",
    iterative_verbosity=1,
    return_sampleset_info=True,
)

Starting community detection iterations


 50%|█████     | 1/2 [01:23<01:23, 83.34s/it]

Iteration 0 completed


100%|██████████| 2/2 [02:58<00:00, 89.24s/it]

Iteration 1 completed


In [12]:
result = res

In [224]:
res[0].sampleset_infos

[SamplesetInfo(dwave_sampleset_info=rec.array((24068., 20., 200.1, 39829.96, 1711.04, 15761.96, 20.58, 16., 16.),
           dtype=[('qpu_sampling_time', '<f8'), ('qpu_anneal_time_per_sample', '<f8'), ('qpu_readout_time_per_sample', '<f8'), ('qpu_access_time', '<f8'), ('qpu_access_overhead_time', '<f8'), ('qpu_programming_time', '<f8'), ('qpu_delay_time_per_sample', '<f8'), ('post_processing_overhead_time', '<f8'), ('total_post_processing_time', '<f8')]), time_measurements=rec.array((0.1773565, 0.0308321, 0.118257),
           dtype=[('find_clique_embedding_time', '<f8'), ('fixed_embedding_composite_time', '<f8'), ('sample_time', '<f8')])),
 SamplesetInfo(dwave_sampleset_info=rec.array((13628., 20., 95.7, 29387.16, 1391.84, 15759.16, 20.58, 88., 88.),
           dtype=[('qpu_sampling_time', '<f8'), ('qpu_anneal_time_per_sample', '<f8'), ('qpu_readout_time_per_sample', '<f8'), ('qpu_access_time', '<f8'), ('qpu_access_overhead_time', '<f8'), ('qpu_programming_time', '<f8'), ('qpu_delay_t

In [262]:
dtype = [si.dwave_sampleset_info for si in res[0].sampleset_infos][0].dtype.descr
dwave_sampleset_infos_0 = np.concatenate([np.array([r], dtype=dtype) for r in [si.dwave_sampleset_info for si in res[0].sampleset_infos]])
dwave_sampleset_infos_0 = dwave_sampleset_infos_0.view(np.recarray)

In [283]:
np.array([np.concatenate([np.array([r], dtype=dtype) for r in [si.dwave_sampleset_info for si in res[run].sampleset_infos]]) for run in range(len(res))], dtype=object)

array([array([(24068., 20., 200.1 , 39829.96, 1711.04, 15761.96, 20.58, 16., 16.),
              (13628., 20.,  95.7 , 29387.16, 1391.84, 15759.16, 20.58, 88., 88.),
              (20728., 20., 166.7 , 36487.96, 1506.04, 15759.96, 20.58,  1.,  1.),
              (12676., 20.,  86.18, 28435.16, 2997.84, 15759.16, 20.58,  1.,  1.),
              (19816., 20., 157.58, 35575.56, 1313.44, 15759.56, 20.58,  1.,  1.),
              (20296., 20., 162.38, 36055.16, 1495.84, 15759.16, 20.58,  1.,  1.),
              (12660., 20.,  86.02, 28419.96, 1273.04, 15759.96, 20.58, 34., 34.),
              (12500., 20.,  84.42, 28259.96, 1336.04, 15759.96, 20.58,  1.,  1.),
              (17362., 20., 133.04, 33121.16,  856.84, 15759.16, 20.58,  1.,  1.),
              (13628., 20.,  95.7 , 29385.96,  819.04, 15757.96, 20.58, 23., 23.),
              (20434., 20., 163.76, 36193.16, 1059.84, 15759.16, 20.58,  1.,  1.),
              (17362., 20., 133.04, 33121.56, 1002.44, 15759.56, 20.58,  1.,  1.),
    

In [288]:
dwave_sampleset_infos = np.array([np.concatenate([np.array([r], dtype=dtype) for r in [si.dwave_sampleset_info for si in res[run].sampleset_infos]]).view(np.recarray) for run in range(len(res))], dtype=object)
dwave_sampleset_infos

array([rec.array([(24068., 20., 200.1 , 39829.96, 1711.04, 15761.96, 20.58, 16., 16.),
                  (13628., 20.,  95.7 , 29387.16, 1391.84, 15759.16, 20.58, 88., 88.),
                  (20728., 20., 166.7 , 36487.96, 1506.04, 15759.96, 20.58,  1.,  1.),
                  (12676., 20.,  86.18, 28435.16, 2997.84, 15759.16, 20.58,  1.,  1.),
                  (19816., 20., 157.58, 35575.56, 1313.44, 15759.56, 20.58,  1.,  1.),
                  (20296., 20., 162.38, 36055.16, 1495.84, 15759.16, 20.58,  1.,  1.),
                  (12660., 20.,  86.02, 28419.96, 1273.04, 15759.96, 20.58, 34., 34.),
                  (12500., 20.,  84.42, 28259.96, 1336.04, 15759.96, 20.58,  1.,  1.),
                  (17362., 20., 133.04, 33121.16,  856.84, 15759.16, 20.58,  1.,  1.),
                  (13628., 20.,  95.7 , 29385.96,  819.04, 15757.96, 20.58, 23., 23.),
                  (20434., 20., 163.76, 36193.16, 1059.84, 15759.16, 20.58,  1.,  1.),
                  (17362., 20., 133.04, 331

In [292]:
dtype = [si.time_measurements for si in res[0].sampleset_infos][0].dtype.descr
time_measurements = np.array([np.concatenate([np.array([r], dtype=dtype) for r in [si.time_measurements for si in res[run].sampleset_infos]]).view(np.recarray) for run in range(len(res))], dtype=object)
time_measurements

array([rec.array([(0.1773565, 0.0308321, 0.118257 ),
                  (0.3025325, 0.0566461, 0.1085171),
                  (0.21881  , 0.0315641, 0.0313734),
                  (0.2612034, 0.0210933, 0.0243663),
                  (0.2666434, 0.0260333, 0.0244937),
                  (0.235935 , 0.029578 , 0.0381151),
                  (0.2194602, 0.0326816, 0.0292636),
                  (0.2763517, 0.0358226, 0.0345125),
                  (0.2058751, 0.0260752, 0.025805 ),
                  (0.1838928, 0.0287553, 0.0464593),
                  (0.1727597, 0.0216226, 0.0276714),
                  (0.1815218, 0.0198324, 0.0204103),
                  (0.1741551, 0.0217112, 0.0206812),
                  (0.2542392, 0.037241 , 0.0455552),
                  (0.1697618, 0.0235122, 0.0260813),
                  (0.2310762, 0.0248809, 0.0225739)],
                 dtype=[('find_clique_embedding_time', '<f8'), ('fixed_embedding_composite_time', '<f8'), ('sample_time', '<f8')]),
       rec.array([(

In [297]:
time_measurements[0].find_clique_embedding_time.sum()

3.531574400010868

In [330]:
results_procesed_dtypes = result.dtype.descr
results_procesed_dtypes.pop()
results_procesed_dtypes.append(("dwave_sampleset_infos", object))
results_procesed_dtypes.append(("time_measurements", object))
results_procesed_dtypes

[('communities', '|O'),
 ('modularity', '<f8'),
 ('time', '<f8'),
 ('division_tree', '|O'),
 ('division_modularities', '|O'),
 ('dwave_sampleset_infos', object),
 ('time_measurements', object)]

In [331]:
results_processed_componenets = [
    res.communities,
    res.modularity,
    res.time,
    res.division_tree,
    res.division_modularities,
    dwave_sampleset_infos,
    time_measurements,
]
results_processed_componenets

[array([list([[0, 4, 17, 18, 19, 23, 29, 40, 42, 57, 71, 95], [1, 8, 12, 32, 38, 45, 47, 51, 55, 60, 61, 66, 74, 78, 93, 98], [24, 26, 28, 37, 59, 63, 86, 92], [68], [10, 13, 25, 27, 30, 31, 41, 48, 49, 64, 70, 82, 91], [2, 7, 11, 14, 15, 21, 34, 39, 44, 52, 62, 67, 73, 94, 99], [3, 5, 6, 50, 56, 65, 69, 76, 89, 90], [9, 16, 22, 35, 43, 72, 77, 79, 81, 84, 85, 96, 97], [20, 33, 36, 46, 53, 54, 58, 75, 80, 83, 87, 88]]),
        list([[0, 4, 17, 18, 19, 23, 29, 40, 42, 57, 71, 95], [1, 8, 12, 32, 38, 45, 47, 51, 55, 60, 61, 66, 74, 78, 93, 98], [24, 26, 28, 37, 59, 63, 86, 92], [56, 65], [10, 13, 25, 27, 30, 31, 41, 48, 49, 64, 82, 91], [3, 5, 6, 50, 68, 69, 76, 89, 90], [2, 7, 11, 14, 15, 21, 34, 39, 44, 52, 62, 67, 73, 94, 99], [9, 16, 22, 35, 43, 72, 77, 79, 81, 84, 85, 96, 97], [70], [20, 33, 36, 46, 53, 54, 58, 75, 80, 83, 87, 88]])],
       dtype=object),
 array([0.7896133 , 0.78252219]),
 array([83.32619762, 95.11350727]),
 array([list([[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,

In [332]:
results_processed = np.rec.fromarrays(
    results_processed_componenets,
    dtype=results_procesed_dtypes,
)

In [342]:
results_processed[0].dwave_sampleset_infos.qpu_access_time.sum()

524900.1599999999